In [1682]:
try:
  import google.colab
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

In [1683]:
!pip install alive_progress
import pandas as pd
import math
import io
from datetime import datetime, time
from alive_progress import alive_bar
import os


# Transcript Mapping

### Set File Names and Variables

Set file names here, the upload boxes will appear under each section of the Read Data section. Please make sure that the real filenames match the file names entered here. 

In [1684]:
# Controls if a sample will be used
# Set to [RANDOM, HEAD, TAIL, NONE]
FILTER = "RANDOM"
SIZE = 1000

# Training Records from SumTotal
filename = "2020-2022 Training Records.xlsx"

# Coursename Mapping Setup
USE_COURSE_MAP = True
CMAP_ORIG = "OLD_CNAME"
CMAP_NEW = "NEW_CNAME"
VERS_OLD = "OLD_VERS"
VERS_NEW = "NEW_VERS"
MAP_PATH = "map.csv"

# Saba Courses
COURSE_PATH = "saba_courses.csv"

# Saba People
PEOPLE_PATH = "saba_people.csv"

### Other Global Variables

In [1685]:
USERNAME = "Employee Number"
COURSEID = "OFFERING"
COURSEVERSION = "COURSE_VERSION"
COMPDATE = "Completion Date"
STATUS = "Completion Status"
COURSETITLE = "Activity Name"
PASSFAIL = "Pass/Fail"
ATTSTATUS = "Attendance Status"
ACT_TYPE = "Activity Type"

FOLDER = "trans_output/"
OUTPUT_PRE = "transcript_out_"
ERRORS_PRE = "transcript_errors_"
MISSING_COURSE = "missing_course_"
MISSING_PEOPLE = "missing_people_"
FILE_EXT = ".csv"
TIME_SUFF = datetime.now().strftime('%Y_%m_%d-%H-%M')

## Read Data

#### Transcript List

In [1686]:
### Note: if using Google Colab, this must be set to the filename of the 
# file you will load. This is a quirk of Colab. 
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  source = pd.read_excel(io.BytesIO(uploaded.get(filename)))
else:
  print("Not Colab")
  source = pd.read_excel(filename)
#source = source[source[ACT_TYPE] == "Course"]

Not Colab


#### Mapping Data

In [1687]:
def map_course(mapping_list, orig_df):
    new_df = orig_df
    for _, map_row in mapping_list.iterrows():
        old_course = map_row[CMAP_ORIG]
        new_course = map_row[CMAP_NEW]
        #new_df[[COURSEID]] = new_df[[COURSEID]].replace(old_course, new_course)
        new_df.replace(old_course, new_course, inplace=True)
        #print(old_course, new_course)
    return new_df

In [1688]:
if USE_COURSE_MAP:
  if IN_COLAB:
    print("Colab")
    uploaded = files.upload()
    course_map = pd.read_csv(io.BytesIO((uploaded.get(MAP_PATH))))
  else:
    print("Not Colab")
    course_map = pd.read_csv(MAP_PATH)

Not Colab


#### Saba Courses

In [1689]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  courses = pd.read_csv(io.BytesIO((uploaded.get(COURSE_PATH))))
else:
  print("Not Colab")
  courses = pd.read_csv(COURSE_PATH)


Not Colab


#### Saba People

In [1690]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  people = pd.read_csv(io.BytesIO((uploaded.get(PEOPLE_PATH))))
else:
  print("Not Colab")
  people = pd.read_csv(PEOPLE_PATH)

Not Colab


## Begin Processing

In [1691]:
# Generate Output Filename
def genFileName(prefix, suffix, extension):
    cwd = os.getcwd()+"/"
    return cwd+FOLDER+str(prefix)+str(suffix)+str(extension)

In [1692]:
# Take Data Sample if Testing
if FILTER == "RANDOM":
    source = source.sample(SIZE)
elif FILTER == "HEAD":
    source = source.head(SIZE)
elif FILTER == "TAIL":
    source = source.tail(SIZE)


In [1693]:
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
32392,56852,"PRESCOTT, Lacey",NaN,Lacey.Prescott@cnl.ca,Waste Management Tech,429.0,WASTE MANAGEMENT OPERATIONS,"HOSBONS, John",ILT Class,PMO-9019,Cost Course - $495/person VIRTUAL Project Comm...,NaN,93.0,Attended,Pass,Completed,2021-12-15,NaT,NaN
25772,56286,"LECLERC, Marc",NaN,Marc.Leclerc@cnl.ca,Grader Operator,8652.0,TRANSPORTATION & FLEET,"HENNESSY, Michael",ILT Class,OSH-3002-C,WS BLOCK - Lift Truck Practical - Counter Balance,NaN,NaN,No-show,NaN,NaN,2022-01-31,NaT,NaN
74830,90456,"VINCENT, Caleb",NaN,caleb.vincent@cnl.ca,Utility Worker,6302.0,CRL DECOMM FACILITIES,"WAKEFIELD, Christopher",Versional,OSH-9104-Online,Cost Course - Trenching Safety Theory,NaN,NaN,Attended,NaN,Completed,2021-11-02,NaT,NaN
817,10664,"WILCOX, Brian",NaN,Brian.Wilcox@cnl.ca,"Director, NPD & WR-1 Reactor D",8626.0,NPD AND WR-1 REACTOR DECOMMISS,"SCHRUDER, Kristan",Versional,900-510000-MCP-015,Harassment and Violance Free Workplace,NaN,NaN,Attended,NaN,Completed,2021-03-22,NaT,NaN
53591,60515,"LEISHMAN, Connie",NaN,Connie.Leishman@cnl.ca,Personnel Security Officer,446.0,WL EMERGENCY SERVICES,"ROSENTRETER, Tim",ILT Class,HR-1005,Virtual Mandatory Harassment and Violence Prev...,NaN,NaN,Attended,Pass,Completed,2021-11-16,NaT,NaN


### Generate Status Table (Optional)

In [1694]:
#print(source.shape[1])
#source[["Completion Status", "Attendance Status", "Pass/Fail"]].groupby(["Pass/Fail","Completion Status", "Attendance Status"], dropna=False, as_index=False).size()

### Update Mapped Course Names

In [1695]:
if USE_COURSE_MAP:
    source = map_course(course_map, source)
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
32392,56852,"PRESCOTT, Lacey",NaN,Lacey.Prescott@cnl.ca,Waste Management Tech,429.0,WASTE MANAGEMENT OPERATIONS,"HOSBONS, John",ILT Class,PMO-9019,Cost Course - $495/person VIRTUAL Project Comm...,NaN,93.0,Attended,Pass,Completed,2021-12-15,NaT,NaN
25772,56286,"LECLERC, Marc",NaN,Marc.Leclerc@cnl.ca,Grader Operator,8652.0,TRANSPORTATION & FLEET,"HENNESSY, Michael",ILT Class,OSH-3002-C,WS BLOCK - Lift Truck Practical - Counter Balance,NaN,NaN,No-show,NaN,NaN,2022-01-31,NaT,NaN
74830,90456,"VINCENT, Caleb",NaN,caleb.vincent@cnl.ca,Utility Worker,6302.0,CRL DECOMM FACILITIES,"WAKEFIELD, Christopher",Versional,OSH-9104-Online,Cost Course - Trenching Safety Theory,NaN,NaN,Attended,NaN,Completed,2021-11-02,NaT,NaN
817,10664,"WILCOX, Brian",NaN,Brian.Wilcox@cnl.ca,"Director, NPD & WR-1 Reactor D",8626.0,NPD AND WR-1 REACTOR DECOMMISS,"SCHRUDER, Kristan",Versional,900-510000-MCP-015,Harassment and Violance Free Workplace,NaN,NaN,Attended,NaN,Completed,2021-03-22,NaT,NaN
53591,60515,"LEISHMAN, Connie",NaN,Connie.Leishman@cnl.ca,Personnel Security Officer,446.0,WL EMERGENCY SERVICES,"ROSENTRETER, Tim",ILT Class,HR-1005,Virtual Mandatory Harassment and Violence Prev...,NaN,NaN,Attended,Pass,Completed,2021-11-16,NaT,NaN


### Read SABA Data

#### Course Data

In [1696]:
# Process Course Data from Saba
course_id = courses[["Course ID", "Version"]]
ids_only = pd.Series(courses["Course ID"].unique())

courses.head()

,Title,Version,Course ID,Unnamed: 3
0,100035-IBM Cognos Analytics Advanced (Cost Cou...,1.0,IT-9050,NaN
1,100035-IBM Cognos Analytics Fundamentals (Cost...,1.0,IT-9049,NaN
2,10 Commandments of Training,1.0,TD-1048,NaN
3,12 GA Shotgun,1.0,SECU-3015,NaN
4,5S Methodology (Cost Course),1.0,PI-9006-ONLINE,NaN


#### Employee Data

In [1697]:
# Process People Data from Saba
people_un = set(people["Username"].astype("int64", errors="ignore").unique())
#people.head()
len(people_un)

3869

### Data Processing Functions

In [1698]:
def procStatus(pass_fail, comp_status, att_status):
    status = -1
    ## Fail
    if pass_fail == "Fail":
        status = 400
    elif (pass_fail == "Pass") & (comp_status == "Not Completed"):
        status = 400
    #elif (math.isnan(pass_fail)) & (math.isnan(comp_status)) & ((att_status == "No-show") | (att_status == "Replaced")):
    #    status = 400
    ## Pass
    elif (pass_fail == "Pass") & (comp_status != "Not Completed"):
        status = 200
    elif (pd.isnull(pass_fail)) & (comp_status == "Completed"):
        status = 200
        #print("no p/f, status is complete")
    elif ( (pd.isnull(pass_fail)) & (pd.isnull(comp_status)) & (att_status == "Attended") ):
        status = 200
        #print("no p/f, no status, attstatus is attended")
    else:
        status = 400
    return status

def checkPerson(unList, person):
    is_in = False
    if str(person) in unList:
        is_in = True
    return is_in

def checkCourse(courseList, courseID, version):
    is_in = False
    replace_version = version
    for _, row in courseList.iterrows():
        #print(courseID, version, row["Course ID"], row["Version"])
        if (courseID == row["Course ID"]):
            #print("Course Match", courseID, version, row["Course ID"], row["Version"])
            if pd.isnull(version):
                replace_version = row["Version"]
                #print("Insert Version", replace_version, version)
            #else:
                #print("Orig Version:", replace_version, version)
            is_in = True
            #print("Match:", courseID, replace_version, version)
    return is_in, replace_version

class TranscriptRow:
    def __init__(self, username, courseID, courseVersion, compDate, status, courseTitle, inCourse, inPerson):
        self.username = username
        self.courseID = courseID.upper()
        self.courseVersion = courseVersion
        self.compDate = compDate
        self.status = status
        self.courseTitle = courseTitle
        self.inCourse = inCourse
        self.inPerson = inPerson


In [1699]:
# Process Data
source_used = source[[USERNAME, COURSEID, COURSEVERSION, COMPDATE, PASSFAIL, STATUS, ATTSTATUS, COURSETITLE]]

transcripts = []

with alive_bar(source_used.shape[0], force_tty=True) as bar_read:
    for index, trans in source_used.iterrows():
        username = trans[USERNAME]
        courseID = trans[COURSEID].upper()
        course_ver = trans[COURSEVERSION]
        comp_date = trans[COMPDATE].strftime('%Y-%m-%d')
        status = procStatus(trans[PASSFAIL], trans[STATUS], trans[ATTSTATUS])
        course_title = trans[COURSETITLE]
        in_course, rep_ver = checkCourse(course_id, courseID, course_ver)
        in_pers = checkPerson(people_un, username)

        #Update Version
        course_ver = rep_ver
        
        trans_entry = TranscriptRow(username, courseID, course_ver, comp_date, status, course_title, in_course, in_pers)
        transcripts.append(trans_entry)
        #print(courseID, type(courseID), courseID in ids_only.values)
        bar_read()
print("Transcripts Processed:", len(transcripts))

|████████████████████████████████████████| 1000/1000 [100%] in 1:06.6 (15.02/s)                                         a: 1:26)  ▆▄▂ 156/1000 [16%] in 13s (12.3/s, eta: 1:07)  ▃▅▇ 180/1000 [18%] in 15s (12.1/s, eta: 1:08)  ▇▇▅ 188/1000 [19%] in 15s (12.2/s, eta: 1:07) ▄▂▂ 229/1000 [23%] in 18s (12.8/s, eta: 1:01)  ▂▂▄ 234/1000 [23%] in 18s (12.8/s, eta: 1:00)  ▁▃▅ 266/1000 [27%] in 20s (13.2/s, eta: 56s) ▃▅▇ 322/1000 [32%] in 24s (13.4/s, eta: 51s) in 24s (13.4/s, eta: 51s)  ▆▄▂ 414/1000 [41%] in 30s (13.8/s, eta: 43s) ▅▃▁ 449/1000 [45%] in 32s (14.1/s, eta: 39s)  ▂▂▄ 543/1000 [54%] in 38s (14.5/s, eta: 32s) (14.6/s, eta: 29s)  ▁▃▅ 602/1000 [60%] in 41s (14.7/s, eta: 27s)  ▆▄▂ 638/1000 [64%] in 44s (14.6/s, eta: 25s)  ▄▆█ 654/1000 [65%] in 45s (14.6/s, eta: 24s)  ▃▁▃ 701/1000 [70%] in 47s (14.8/s, eta: 20s) ▂▂▄ 731/1000 [73%] in 49s (14.8/s, eta: 18s) ▃▅▇ 763/1000 [76%] in 52s (14.8/s, eta: 16s)  765/1000 [76%] in 52s (14.8/s, eta: 16s)  ▆█▆ 823/1000 [82%] in 55s (14.8/s, eta: 12s)  ▄

### Print to File Outputs

In [1700]:
# Process a Transcript Row
def write_row(df_output, trans, test=False, subset="ALL"):
    #df = df_output
    
    inCourse = trans.inCourse
    inPerson = trans.inPerson

    tmp_dict = {
        "LEARNER": trans.username,
        "COURSE": trans.courseID,
        "COURSE_VERSION": trans.courseVersion,
        "COMPLETION_DATE": trans.compDate,
        "COMPLETION_STATUS": trans.status,
        "CREATE_ADHOC_COURSE": "FALSE",
        "COURSE_TITLE": trans.courseTitle
    }
    if test == True:
        tmp_dict = {
            "LEARNER": trans.username,
            "COURSE": trans.courseID,
            "COURSE_VERSION": trans.courseVersion,
            "COMPLETION_DATE": trans.compDate,
            "COMPLETION_STATUS": trans.status,
            "CREATE_ADHOC_COURSE": "FALSE",
            "COURSE_TITLE": trans.courseTitle,
            "IN_COURSE": trans.inCourse,
            "IN_PERSON": trans.inPerson
        }
    if subset == "ALL":
        df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "CLEAN":
        if ( (inCourse == 1) & (inPerson == 1) ):
            df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "MISSING":
        if ( (inCourse == 0) | (inPerson == 0) ):
            tmp_dict["COURSE_GOOD"] = bool(inCourse)
            tmp_dict["PERSON_GOOD"] = bool(inPerson)
            df_output = df_output.append(tmp_dict, ignore_index=True)
    
    return df_output
    

In [1701]:
# Output Header Values
out_header = [
    "LEARNER",
    "COURSE",
    "COURSE_VERSION",
    "COMPLETION_DATE",
    "COMPLETION_STATUS",
    "CREATE_ADHOC_COURSE",
    "COURSE_TITLE"
]

### Generate Saba Import Data Table and Write to File

In [1704]:
out = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out = write_row(out, ts, False, "CLEAN")
        bar()
out["ID"] = out.index
out.sample(20).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.9s (537.76/s)                                          


,177,42,499,421,90,429,281,431,428,54,327,221,467,95,1,80,530,106,515,525
LEARNER,50880,60792,56450,61474,92095,57067,92523,20674,59571,55034,91962,56797,56392,91102,56286,60923,91223,91885,60422,60325
COURSE,TD-2000-WL,HR-1005,OPEX-1004,FIRE-1183-CRL,SECU-1010,HR-1005,FIRE-1001-ONLINE,TD-1032,HR-1005,SECU-2029,OSH-1007,OSH-1004,TD-1035,OSH-9046-ONLINE,OSH-3002-C,HR-1005,MS-3001,PHAI-1004,OSH-1004-VIRTUAL,WL-305
COURSE_VERSION,1.0,1.0,1.0,1.0,1.0,1.0,4.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
COMPLETION_DATE,2021-09-21,2021-11-18,2021-12-13,2021-05-31,2021-03-25,2021-11-26,2021-12-14,2021-06-03,2021-11-04,2022-03-14,2021-02-26,2022-03-28,2022-02-17,2021-07-19,2022-01-31,2021-11-09,2021-03-03,2022-03-17,2022-01-31,2021-01-25
COMPLETION_STATUS,200,200,200,200,200,200,200,200,200,200,400,200,400,200,400,200,200,200,200,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,GET General Employee Training,Mandatory Harassment and Violence Prevention T...,Fact Finding,Rope Rescue: System Safety Checks,Equipment Familiarization,Virtual Mandatory Harassment and Violence Prev...,Fire Prevention,Responsibilities In Supervising Employees (RIS...,Mandatory Harassment and Violence Prevention T...,Security Awareness Refresher,VIRTUAL Asbestos Module 6E,Lock Out / Tag Out,RISE Field Essentials,Hearing Conservation,WS BLOCK - Lift Truck Practical - Counter Balance,Virtual Mandatory Harassment and Violence Prev...,FD/ER - Integrated Work Control Practical Appl...,Basic Principles Underlying Situational Awareness,Lock Out / Tag Out (Virtual),Stop Work
ID,177,42,499,421,90,429,281,431,428,54,327,221,467,95,1,80,530,106,515,525


In [ ]:
if IN_COLAB:
    out.to_csv("out_transcript.csv", index=False)
else:
    out.to_csv(genFileName(OUTPUT_PRE, TIME_SUFF, FILE_EXT), index=False)

In [ ]:
# OLD OUTPUT
#print(out.shape)
#out.to_csv("out_transcript.csv", index=False)
#out.to_csv(genFileName(OUTPUT_PRE, TIME_SUFF, FILE_EXT), index=False)

#### Write Failiures to File

In [ ]:
out_err = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out_err = write_row(out_err, ts, False, "MISSING")
        bar()
out_err["ID"] = out_err.index
out_err.sample(20).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.5s (670.82/s)                                          s, eta: 1s) 


,204,292,346,210,264,405,321,384,154,37,336,386,233,133,363,88,112,327,220,208
LEARNER,59286,27606,60409,91266,58693,54886,60035,92106,91214,91992,60038,CONT-3099,92353,59114,52815,60800,61035,20247,59370,CONT-2844
COURSE,FIRE-1001-ONLINE (REV 4.5),ENG-1069-ONLINE (REV 0.1),OSH-9045-ONLINE (REV 1),OSH-9045-ONLINE (REV 1),LGL-1003-ONLINE,SECU-2008-ONLINE (REV 1.2),OSH-1006,PROC-1019,OSH-1004-ONLINE,RP-G4-ONLINE,WL-315,MS-1004-WL,QUAL-1013-ONLINE (REV 3),OSH-9045-ONLINE (REV 1),FD-2001-B204,SECU-2008-ONLINE (REV 1.2),OSH-1001-ONLINE (REV 1.2),SECU-2008-ONLINE (REV 1.3),OSH-9045-ONLINE (REV 1),WL-120
COURSE_VERSION,4.5,0.1,1.0,1.0,NaN,1.2,NaN,1.0,NaN,NaN,1.0,1.0,3.0,1.0,NaN,1.2,1.2,1.3,1.0,1.0
COMPLETION_DATE,2021-06-21,2022-03-29,2021-10-13,2021-07-16,2021-12-10,2021-05-06,2021-02-10,2021-05-20,2021-01-14,2021-07-12,2021-03-11,2022-01-10,2021-11-01,2021-11-04,2021-02-15,2021-05-19,2021-02-18,2021-06-03,2021-07-26,2021-05-03
COMPLETION_STATUS,200,200,200,200,200,200,200,200,200,200,400,200,400,200,200,200,200,200,200,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Fire Prevention (4.5),Fume Hood Safety (0.1),Cost Course - Hand Safety (1.0),Cost Course - Hand Safety (1.0),Values and Ethics at CNL,Security Awareness Refresher (1.2),Confined Space Entry,VIRTUAL CNL Contractor Management Overview,Lock Out Tag Out Exam,Group 4 Radiation Protection Training,Respectful Work Place,Integrated Work Control 5 Core Elements,FME Awareness CRL (3.0),Cost Course - Hand Safety (1.0),B204 Building Familiarization Checkout,Security Awareness Refresher (1.2),Crane - Safe Indoor Hoist - Theory (1.2),Security Awareness Refresher (1.3),Cost Course - Hand Safety (1.0),Whiteshell Site Orientation
COURSE_GOOD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
PERSON_GOOD,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
ID,204,292,346,210,264,405,321,384,154,37,336,386,233,133,363,88,112,327,220,208


In [ ]:
if IN_COLAB:
    out_err.to_csv("out_errors.csv", index=False)
else:
    out_err.to_csv(genFileName(ERRORS_PRE, TIME_SUFF, FILE_EXT), index=False)

In [ ]:
# OLD OUTPUT
#print(out_err.shape)
#out_err.to_csv("out_errors.csv", index=False)
#out_err.to_csv(genFileName(ERRORS_PRE, TIME_SUFF, FILE_EXT), index=False)

### Generate Lists of Missing Courses and People

Generate list of course IDs and usernames that are in the transcript data, but do not exist in Saba. We know these will fail since the requisite data does not exist in Saba. 

In [ ]:
course_need = out_err[out_err["COURSE_GOOD"] == False]
person_need = out_err[out_err["PERSON_GOOD"] == False]

# Old Output
#pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
#pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")
#pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv(genFileName(MISSING_COURSE, TIME_SUFF, FILE_EXT))
#pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv(genFileName(MISSING_PEOPLE, TIME_SUFF, FILE_EXT))

In [ ]:
if IN_COLAB:
    pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
    pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")
else:
    pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv(genFileName(MISSING_COURSE, TIME_SUFF, FILE_EXT))
    pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv(genFileName(MISSING_PEOPLE, TIME_SUFF, FILE_EXT))